In [1]:
import pandas as pd
import asyncio
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import contractions
from googletrans import Translator
import json
import os
from datetime import datetime, timedelta

import nest_asyncio
nest_asyncio.apply()


In [2]:
# Init Bahasa Indonesia stopwords
stpwds_id = stopwords.words('indonesian')

# init Sastrawi stemmer for Bahasa Indonesia
factory = StemmerFactory()
stemmer_id = factory.create_stemmer()

In [3]:
translator = Translator()

In [4]:
# Load the slang dictionary
with open('slang_bank.json', 'r') as file:
    slang_dict = json.load(file)

In [5]:
async def text_preprocessing_id(text):
    
    # Translate to Bahasa Indonesia
    
    try:
        translated = await translator.translate(text, src='auto', dest='id')
        text = translated.text
    except Exception as e:
        print(f"Translation failed: {e}")
        return None  # Drop text if translation fails
    
    # Expand contractions
    text = contractions.fix(text)

    # Case folding
    text = text.lower()

    # Mention removal
    text = re.sub(r"@[A-Za-z0-9_]+", " ", text)

    # Hashtag removal
    text = re.sub(r"#[A-Za-z0-9_]+", " ", text)

    # Newline removal (\n)
    text = re.sub(r"\\n", " ", text)

    # Whitespace removal
    text = text.strip()

    # URL removal
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"www.\S+", " ", text)

    # Non-letter removal (retain apostrophes)
    text = re.sub(r"[^A-Za-z\s']", " ", text)
    
    # repeat letter removal
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # Tokenization
    tokens = word_tokenize(text)
    
    # Slang words replacement
    tokens = [slang_dict[word] if word in slang_dict else word for word in tokens]

    # Stopwords removal
    tokens = [word for word in tokens if word not in stpwds_id or word in {"tidak", "belum", "jangan", "enggak", "gak"}]

    # Stemming (using Sastrawi for Bahasa Indonesia)
    tokens = [stemmer_id.stem(word) for word in tokens]

    # Combine tokens
    text = ' '.join(tokens)

    return text


In [6]:
# samples
sample_reviews_id = [
    "Pengiriman sangat lambat dan pegawai tidak ramah. #mengecewakan",
    "Pelayanan sangat baik! Cepat dan murah. Sangat direkomendasikan.",
    "Saya harus menunggu 3 jam di gudang. @perusahaan tidak bisa diterima!",
    "Can you be like the others? I'm very disappointed with your services.",
    "Kntl", 
    """He! sby-mlg perjalanan mosok iyo 3 hari. mau dikirim dari jam 7 pagi. jarak warehouse kerumah naik sepeda motor 5 menit!!!! yo wes seharian gak ono kabar
ngerti gak, expedisi lain dikasih kontak kurirnya! ini opooo -_-
ujung e gagal kirim !
dari jaman gak ada shopee tokped sek bermasalah ae
tutup ae gak akan bertahan yoan mending wahana/jnt terutama tiki /shopee express

shopee ngasih kompensasi gak tepat waktu gara2 kerjomu ngerti gak
kerja nyari berkah iki nyari dihujat orang kok yo iso makan keringat hasil grundelan e orang -_-

bubar ae gak guna

wes lah besok2 dapat jne mending dibatalno ae -_-
    """,
    "Kntl asu, pukimak, babi, bubar ae cok. Gak guna!",
    "Kulo kok rada kecewa kaliyan pelayanan kurir punika. Paket sing mesthine tekan sesuk, malah telat tekan nganti 3 dinten. Sing langkung parah, paket punika mboten dipunparingi kabar punapa-punapa, lan kulo kedah ngontak layanan pelanggan piyambak.",
    "Apo jo kurir kini? Jan langsuang kasiah barek! Paket nan awak kirim tibo-tibo ado di mana, lapau indak jelas, tarang tuan-tuan indak ado kabar! Jan ambo baliak-baliak mangecek ka call center, malah awak pulo nan mangambek! Cakak bana pelayanannyo, indak ado adab! Awak bayia mahal-mahal, alah kironyo mah pelayanan macam indak ado otak! Tolong lah, jangan macam ini lai ke depan",
    "JNE pungo, hana utak, hanjeut meupike gob rugo, lage bui…😂😂",
    "Jne paleh lehoho kah me barang , ka pike hayeu tat kah nyan. Ok me jih keuh. Kon kah manteng hayeu! lhe lom expedisi selain kah."
]

# make dataframe
df_test_id = pd.DataFrame(sample_reviews_id, columns=["review"])

# 

In [7]:
# apply text preprocessing
# df_test_id["processed_review"] = df_test_id["review"].apply(text_preprocessing_id)

In [8]:
# Async loop to process reviews
async def process_reviews():
    df_test_id["processed_review"] = await asyncio.gather(
        *[text_preprocessing_id(text) for text in df_test_id["review"]]
    )

In [9]:
await process_reviews()

In [10]:
# Display the original and processed reviews
df_test_id

,review,processed_review
0,Pengiriman sangat lambat dan pegawai tidak ram...,kirim lambat pegawai tidak ramah
1,Pelayanan sangat baik! Cepat dan murah. Sangat...,layan cepat murah rekomendasi
2,Saya harus menunggu 3 jam di gudang. @perusaha...,tunggu jam gudang tidak terima
3,Can you be like the others? I'm very disappoin...,kecewa layan
4,Kntl,kontol
5,He! sby-mlg perjalanan mosok iyo 3 hari. mau d...,he sby mlg jalan ya kirim jam pagi jarak wareh...
6,"Kntl asu, pukimak, babi, bubar ae cok. Gak guna!",kontol asu pukimak babi tutup cok tidak guna
7,Kulo kok rada kecewa kaliyan pelayanan kurir p...,kecewa jasa kurir paket besok tunda buruk pake...
8,Apo jo kurir kini? Jan langsuang kasiah barek!...,kurir jangan langsung jatuh cinta paket kirim ...
9,"JNE pungo, hana utak, hanjeut meupike gob rugo...",jne gila tidak otak tidak mikir orang kalah ka...


In [11]:
pd.set_option('display.max_colwidth', None)

# Display the original and processed reviews
df_test_id

,review,processed_review
0,Pengiriman sangat lambat dan pegawai tidak ramah. #mengecewakan,kirim lambat pegawai tidak ramah
1,Pelayanan sangat baik! Cepat dan murah. Sangat direkomendasikan.,layan cepat murah rekomendasi
2,Saya harus menunggu 3 jam di gudang. @perusahaan tidak bisa diterima!,tunggu jam gudang tidak terima
3,Can you be like the others? I'm very disappointed with your services.,kecewa layan
4,Kntl,kontol
5,"He! sby-mlg perjalanan mosok iyo 3 hari. mau dikirim dari jam 7 pagi. jarak warehouse kerumah naik sepeda motor 5 menit!!!! yo wes seharian gak ono kabar\nngerti gak, expedisi lain dikasih kontak kurirnya! ini opooo -_-\nujung e gagal kirim !\ndari jaman gak ada shopee tokped sek bermasalah ae\ntutup ae gak akan bertahan yoan mending wahana/jnt terutama tiki /shopee express\n\nshopee ngasih kompensasi gak tepat waktu gara2 kerjomu ngerti gak\nkerja nyari berkah iki nyari dihujat orang kok yo iso makan keringat hasil grundelan e orang -_-\n\nbubar ae gak guna\n\nwes lah besok2 dapat jne mending dibatalno ae -_-\n",he sby mlg jalan ya kirim jam pagi jarak warehouse rumah sepeda motor menit ya hari tidak kabar erti tidak expedisi kasih kontak kurir ujung e gagal kirim jaman tidak shopee tokped sek masalah tutup tidak tahan yoan mending wahana jnt tik shopee express shopee ngasih kompensasi tidak gara kerjomu erti tidak kerja nyari berkah nyari hujat orang ya iso makan keringat hasil grundelan e orang tutup tidak besok jne mending dibatalno
6,"Kntl asu, pukimak, babi, bubar ae cok. Gak guna!",kontol asu pukimak babi tutup cok tidak guna
7,"Kulo kok rada kecewa kaliyan pelayanan kurir punika. Paket sing mesthine tekan sesuk, malah telat tekan nganti 3 dinten. Sing langkung parah, paket punika mboten dipunparingi kabar punapa-punapa, lan kulo kedah ngontak layanan pelanggan piyambak.",kecewa jasa kurir paket besok tunda buruk paket tidak pemberitahuan hubung layan langgan
8,"Apo jo kurir kini? Jan langsuang kasiah barek! Paket nan awak kirim tibo-tibo ado di mana, lapau indak jelas, tarang tuan-tuan indak ado kabar! Jan ambo baliak-baliak mangecek ka call center, malah awak pulo nan mangambek! Cakak bana pelayanannyo, indak ado adab! Awak bayia mahal-mahal, alah kironyo mah pelayanan macam indak ado otak! Tolong lah, jangan macam ini lai ke depan",kurir jangan langsung jatuh cinta paket kirim kafe tidak tidak kabar jangan biar bicara call center kau blokir layan buruk tidak sopan santun bayar mahal mah layan tidak otak tolong jangan laku
9,"JNE pungo, hana utak, hanjeut meupike gob rugo, lage bui…😂😂",jne gila tidak otak tidak mikir orang kalah kaya tahan


---

# testing with sample scrapped

In [12]:
# load the dataset
df = pd.read_csv('test.csv')

In [13]:
df

,ID,Name,Rating,Reviews,Date
0,0,Irynt S,0,"Kurirnya ga sopan. Kronologisnya, kurir nelpon sy 6 kali dengan rentang waktu 1 menit tiap telepon, sy saat itu tidak lg pegang hp krn sedang kerja. Waktu sy telpon balik 10 menit kemudian katanya tidak bsa antar paket lagi krn dia tidak …",6 bulan lalu
1,1,Masrizal Mahmud,0,"Muak dengan kinerja JNE Banda Aceh. Di tracking disebutkan barang sedang diantar kurir, dua hari berturut2. Waktu kantornya saya datangi, tiba2 secara ajaib barang keluar dari gudang dalam 1 menit. Trus yg diantar kurir apa? Tuyul? Kejadian …",4 tahun lalu
2,2,Sartivina Br ginting,0,"Jne sampah, kerjanya apa si cmn tgl ngantar kan brg aja di tahan"" ,udh express juga,karna org perlu cepat ini ditahan tahan ,udah la paketnya isinya mahal",6 bulan lalu
3,3,Adex Sari,0,"Di trucking paket sudah diterima,, foto alamat ntah dmna"" gak sesuai sama alamat yg sudah saya terakan..\nPadahal alamat sudah jelas kantor daihatsu …",9 bulan lalu
4,4,Tasya Rania Insyara,0,gak jelas banget jam operasionalnya\ndi maps dibikin buka setiap hari\ndatang di hari sabtu dan minggu ternyata gak buka …,10 bulan lalu
...,...,...,...,...,...
969,354,Sultan Andrews Abieza,3,No review found,5 tahun lalu
970,355,Andri Awan,4,No review found,5 tahun lalu
971,356,Muhammad Taufik,3,No review found,5 tahun lalu
972,357,cici nurrahmah,5,No review found,6 tahun lalu


In [14]:
df['Rating'].value_counts()

Rating
1    388
0    256
5    168
3     74
4     56
2     32
Name: count, dtype: int64

In [15]:
# devide bad and good reviews
## bad reviews = 1 - 3 stars
## good reviews = 4 - 5 stars

bad_reviews_df = df[df['Rating'] <= 3]
good_reviews_df = df[df['Rating'] > 3]

In [16]:
df['Reviews'][df['Reviews'] == 'No review found'].count()

248

In [17]:
bad_reviews_df

,ID,Name,Rating,Reviews,Date
0,0,Irynt S,0,"Kurirnya ga sopan. Kronologisnya, kurir nelpon sy 6 kali dengan rentang waktu 1 menit tiap telepon, sy saat itu tidak lg pegang hp krn sedang kerja. Waktu sy telpon balik 10 menit kemudian katanya tidak bsa antar paket lagi krn dia tidak …",6 bulan lalu
1,1,Masrizal Mahmud,0,"Muak dengan kinerja JNE Banda Aceh. Di tracking disebutkan barang sedang diantar kurir, dua hari berturut2. Waktu kantornya saya datangi, tiba2 secara ajaib barang keluar dari gudang dalam 1 menit. Trus yg diantar kurir apa? Tuyul? Kejadian …",4 tahun lalu
2,2,Sartivina Br ginting,0,"Jne sampah, kerjanya apa si cmn tgl ngantar kan brg aja di tahan"" ,udh express juga,karna org perlu cepat ini ditahan tahan ,udah la paketnya isinya mahal",6 bulan lalu
3,3,Adex Sari,0,"Di trucking paket sudah diterima,, foto alamat ntah dmna"" gak sesuai sama alamat yg sudah saya terakan..\nPadahal alamat sudah jelas kantor daihatsu …",9 bulan lalu
4,4,Tasya Rania Insyara,0,gak jelas banget jam operasionalnya\ndi maps dibikin buka setiap hari\ndatang di hari sabtu dan minggu ternyata gak buka …,10 bulan lalu
...,...,...,...,...,...
967,352,Irvan Verdian,3,No review found,5 tahun lalu
968,353,Mustafa Kamal,2,No review found,5 tahun lalu
969,354,Sultan Andrews Abieza,3,No review found,5 tahun lalu
971,356,Muhammad Taufik,3,No review found,5 tahun lalu


In [18]:
# drop rows with 'No review found'
bad_reviews_df = bad_reviews_df[bad_reviews_df['Reviews'] != 'No review found']

In [19]:
bad_reviews_df

,ID,Name,Rating,Reviews,Date
0,0,Irynt S,0,"Kurirnya ga sopan. Kronologisnya, kurir nelpon sy 6 kali dengan rentang waktu 1 menit tiap telepon, sy saat itu tidak lg pegang hp krn sedang kerja. Waktu sy telpon balik 10 menit kemudian katanya tidak bsa antar paket lagi krn dia tidak …",6 bulan lalu
1,1,Masrizal Mahmud,0,"Muak dengan kinerja JNE Banda Aceh. Di tracking disebutkan barang sedang diantar kurir, dua hari berturut2. Waktu kantornya saya datangi, tiba2 secara ajaib barang keluar dari gudang dalam 1 menit. Trus yg diantar kurir apa? Tuyul? Kejadian …",4 tahun lalu
2,2,Sartivina Br ginting,0,"Jne sampah, kerjanya apa si cmn tgl ngantar kan brg aja di tahan"" ,udh express juga,karna org perlu cepat ini ditahan tahan ,udah la paketnya isinya mahal",6 bulan lalu
3,3,Adex Sari,0,"Di trucking paket sudah diterima,, foto alamat ntah dmna"" gak sesuai sama alamat yg sudah saya terakan..\nPadahal alamat sudah jelas kantor daihatsu …",9 bulan lalu
4,4,Tasya Rania Insyara,0,gak jelas banget jam operasionalnya\ndi maps dibikin buka setiap hari\ndatang di hari sabtu dan minggu ternyata gak buka …,10 bulan lalu
...,...,...,...,...,...
843,228,Solong To YOU BABE,2,Tidak buruk,7 tahun lalu
844,229,Novan 87,1,Pelayanan baaaddddd.....,7 tahun lalu
845,230,Bachtiar Arrahman,2,Respon lambat,2 tahun lalu
847,232,Syifa Alisha,3,Hmmmmm,6 tahun lalu


In [20]:
bad_reviews_df_prepo = bad_reviews_df.copy()
async def preprocess_reviews():
    bad_reviews_df_prepo['processed_review'] = await asyncio.gather(
        *[text_preprocessing_id(review) for review in bad_reviews_df_prepo['Reviews']]
    )

await preprocess_reviews()

In [21]:
bad_reviews_df_prepo

,ID,Name,Rating,Reviews,Date,processed_review
0,0,Irynt S,0,"Kurirnya ga sopan. Kronologisnya, kurir nelpon sy 6 kali dengan rentang waktu 1 menit tiap telepon, sy saat itu tidak lg pegang hp krn sedang kerja. Waktu sy telpon balik 10 menit kemudian katanya tidak bsa antar paket lagi krn dia tidak …",6 bulan lalu,kurir tidak sopan kronologis kurir nelpon sy kali rentang menit telepon sy tidak lg pegang hp krn kerja sy telpon menit tidak bsa paket krn tidak
1,1,Masrizal Mahmud,0,"Muak dengan kinerja JNE Banda Aceh. Di tracking disebutkan barang sedang diantar kurir, dua hari berturut2. Waktu kantornya saya datangi, tiba2 secara ajaib barang keluar dari gudang dalam 1 menit. Trus yg diantar kurir apa? Tuyul? Kejadian …",4 tahun lalu,muak kerja jne banda aceh tracking barang antar kurir kantor datang ajaib barang gudang menit trus yg antar kurir tuyul jadi
2,2,Sartivina Br ginting,0,"Jne sampah, kerjanya apa si cmn tgl ngantar kan brg aja di tahan"" ,udh express juga,karna org perlu cepat ini ditahan tahan ,udah la paketnya isinya mahal",6 bulan lalu,jne sampah kerja si cmn tgl ngantar brg tahan udh express karna org cepat tahan tahan la paket isi mahal
3,3,Adex Sari,0,"Di trucking paket sudah diterima,, foto alamat ntah dmna"" gak sesuai sama alamat yg sudah saya terakan..\nPadahal alamat sudah jelas kantor daihatsu …",9 bulan lalu,trucking paket terima foto alamat ntah dmna tidak sesuai alamat yg tera alamat kantor daihatsu
4,4,Tasya Rania Insyara,0,gak jelas banget jam operasionalnya\ndi maps dibikin buka setiap hari\ndatang di hari sabtu dan minggu ternyata gak buka …,10 bulan lalu,tidak banget jam operasional maps bikin buka sabtu minggu tidak buka
...,...,...,...,...,...,...
843,228,Solong To YOU BABE,2,Tidak buruk,7 tahun lalu,tidak buruk
844,229,Novan 87,1,Pelayanan baaaddddd.....,7 tahun lalu,layan bad
845,230,Bachtiar Arrahman,2,Respon lambat,2 tahun lalu,respon lambat
847,232,Syifa Alisha,3,Hmmmmm,6 tahun lalu,hm


---

In [22]:
# Define the folder path
folder_path = './reviews'

# Initialize an empty list to store dataframes
dataframes = []

# Iterate over all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        # Optionally add a column to track the source file
        df['Source'] = file_name
        # Append the dataframe to the list
        dataframes.append(df)

# Combine all dataframes into a single dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined dataframe to a CSV file
output_path = 'compiled_reviews.csv'
combined_df.to_csv(output_path, index=False)

print(f"Combined dataframe saved to {output_path}")


Combined dataframe saved to compiled_reviews.csv


In [23]:
final_df = pd.read_csv('compiled_reviews.csv')

In [24]:
final_df

,Rating,Reviews,Date,Company,Province,Source
0,5,"Located at the Padang Old City area, just around the corner from Siti Nurbaya Bridge across Batang Anai River. JNE is one of many delivery services in Indonesia. Giving good services, the business entity continues to grow.",9 months ago,JNE,Aceh,JNE_Aceh.csv
1,1,"Bad service given to customers, my own experience taking 1 day shipment service and had to pay quite big amount of money. I asked over and over to make sure my package arrived in 1 day as it contained food and JNE staffs answer so sure it …",4 years ago,JNE,Aceh,JNE_Aceh.csv
2,5,"Nice Place, they're working on rebuild their office, great people and services",3 years ago,JNE,Aceh,JNE_Aceh.csv
3,5,Good delivery service for customers in this country.,2 years ago,JNE,Aceh,JNE_Aceh.csv
4,5,Good service,3 years ago,JNE,Aceh,JNE_Aceh.csv
...,...,...,...,...,...,...
2786,5,No review found,NaN,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2787,5,No review found,NaN,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2788,5,No review found,NaN,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2789,5,No review found,NaN,JNE,Sumatera Utara,JNE_Sumatera Utara.csv


In [25]:
# remove rows with 'No review found'
final_df = final_df[final_df['Reviews'] != 'No review found']

In [26]:
final_df

,Rating,Reviews,Date,Company,Province,Source
0,5,"Located at the Padang Old City area, just around the corner from Siti Nurbaya Bridge across Batang Anai River. JNE is one of many delivery services in Indonesia. Giving good services, the business entity continues to grow.",9 months ago,JNE,Aceh,JNE_Aceh.csv
1,1,"Bad service given to customers, my own experience taking 1 day shipment service and had to pay quite big amount of money. I asked over and over to make sure my package arrived in 1 day as it contained food and JNE staffs answer so sure it …",4 years ago,JNE,Aceh,JNE_Aceh.csv
2,5,"Nice Place, they're working on rebuild their office, great people and services",3 years ago,JNE,Aceh,JNE_Aceh.csv
3,5,Good delivery service for customers in this country.,2 years ago,JNE,Aceh,JNE_Aceh.csv
4,5,Good service,3 years ago,JNE,Aceh,JNE_Aceh.csv
...,...,...,...,...,...,...
2691,1,Lambaaaaaat!!,NaN,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2692,5,👌👌👌 …,NaN,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2693,5,24 jam,NaN,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2694,4,Ya,NaN,JNE,Sumatera Utara,JNE_Sumatera Utara.csv


In [27]:
final_df.isnull().sum()

Rating         0
Reviews        0
Date        1632
Company        0
Province       0
Source         0
dtype: int64

In [28]:
# fill nan values in Date column with "6 months ago"
final_df['Date'] = final_df['Date'].fillna('6 months ago')

C:\Users\Niko Amrullah Hakam\AppData\Local\Temp\ipykernel_508\7627352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Date'] = final_df['Date'].fillna('6 months ago')


In [29]:
final_df

,Rating,Reviews,Date,Company,Province,Source
0,5,"Located at the Padang Old City area, just around the corner from Siti Nurbaya Bridge across Batang Anai River. JNE is one of many delivery services in Indonesia. Giving good services, the business entity continues to grow.",9 months ago,JNE,Aceh,JNE_Aceh.csv
1,1,"Bad service given to customers, my own experience taking 1 day shipment service and had to pay quite big amount of money. I asked over and over to make sure my package arrived in 1 day as it contained food and JNE staffs answer so sure it …",4 years ago,JNE,Aceh,JNE_Aceh.csv
2,5,"Nice Place, they're working on rebuild their office, great people and services",3 years ago,JNE,Aceh,JNE_Aceh.csv
3,5,Good delivery service for customers in this country.,2 years ago,JNE,Aceh,JNE_Aceh.csv
4,5,Good service,3 years ago,JNE,Aceh,JNE_Aceh.csv
...,...,...,...,...,...,...
2691,1,Lambaaaaaat!!,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2692,5,👌👌👌 …,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2693,5,24 jam,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv
2694,4,Ya,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv


In [30]:
# let's translate the Date column to bahasa indonesia first
## def func to tran
async def translate_to_indonesian_async(text):
    try:
        translated = await translator.translate(text, src='auto', dest='id')
        return translated.text
    except Exception as e:
        print(f"Translation error: {e}")
        return text

In [31]:
async def translate_column(df, column_name):
    tasks = [translate_to_indonesian_async(text) for text in df[column_name]]
    translated_texts = await asyncio.gather(*tasks)
    df[f"{column_name}_translated"] = translated_texts

In [32]:
await translate_column(final_df, "Date")

C:\Users\Niko Amrullah Hakam\AppData\Local\Temp\ipykernel_508\3877582011.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{column_name}_translated"] = translated_texts


In [33]:
final_df

,Rating,Reviews,Date,Company,Province,Source,Date_translated
0,5,"Located at the Padang Old City area, just around the corner from Siti Nurbaya Bridge across Batang Anai River. JNE is one of many delivery services in Indonesia. Giving good services, the business entity continues to grow.",9 months ago,JNE,Aceh,JNE_Aceh.csv,9 bulan yang lalu
1,1,"Bad service given to customers, my own experience taking 1 day shipment service and had to pay quite big amount of money. I asked over and over to make sure my package arrived in 1 day as it contained food and JNE staffs answer so sure it …",4 years ago,JNE,Aceh,JNE_Aceh.csv,4 tahun yang lalu
2,5,"Nice Place, they're working on rebuild their office, great people and services",3 years ago,JNE,Aceh,JNE_Aceh.csv,3 tahun yang lalu
3,5,Good delivery service for customers in this country.,2 years ago,JNE,Aceh,JNE_Aceh.csv,2 tahun yang lalu
4,5,Good service,3 years ago,JNE,Aceh,JNE_Aceh.csv,3 tahun yang lalu
...,...,...,...,...,...,...,...
2691,1,Lambaaaaaat!!,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu
2692,5,👌👌👌 …,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu
2693,5,24 jam,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu
2694,4,Ya,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu


In [34]:
# let's attempt to transform the date
## def the func

def parse_time_ago(time_ago):
    current_time = datetime.now()
    
    # Match cases with explicit numbers
    match = re.match(r"(\d+)\s+(\w+)", time_ago)
    if match:
        value, unit = int(match.group(1)), match.group(2)
    else:
        # Handle "sehari", "seminggu", "sebulan", "setahun" cases
        if time_ago.startswith('se'):
            value, unit = 1, time_ago.split()[0][2:]  # Extract the unit after "se"
        else:
            return None  # If it doesn't match any known format, return None
    
    # Map units to timedelta
    if unit.startswith('menit'):
        return current_time - timedelta(minutes=value)
    elif unit.startswith('jam'):
        return current_time - timedelta(hours=value)
    elif unit.startswith('hari'):
        return current_time - timedelta(days=value)
    elif unit.startswith('minggu'):
        return current_time - timedelta(weeks=value)
    elif unit.startswith('bulan'):
        # Approximate 1 month = 30 days
        return current_time - timedelta(days=value * 30)
    elif unit.startswith('tahun'):
        # Approximate 1 year = 365 days
        return current_time - timedelta(days=value * 365)
    return None


In [35]:
final_df['parsed_date'] = final_df['Date_translated'].apply(parse_time_ago)

C:\Users\Niko Amrullah Hakam\AppData\Local\Temp\ipykernel_508\2538609799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['parsed_date'] = final_df['Date_translated'].apply(parse_time_ago)


In [36]:
final_df

,Rating,Reviews,Date,Company,Province,Source,Date_translated,parsed_date
0,5,"Located at the Padang Old City area, just around the corner from Siti Nurbaya Bridge across Batang Anai River. JNE is one of many delivery services in Indonesia. Giving good services, the business entity continues to grow.",9 months ago,JNE,Aceh,JNE_Aceh.csv,9 bulan yang lalu,2024-04-23 21:52:17.056259
1,1,"Bad service given to customers, my own experience taking 1 day shipment service and had to pay quite big amount of money. I asked over and over to make sure my package arrived in 1 day as it contained food and JNE staffs answer so sure it …",4 years ago,JNE,Aceh,JNE_Aceh.csv,4 tahun yang lalu,2021-01-19 21:52:17.056259
2,5,"Nice Place, they're working on rebuild their office, great people and services",3 years ago,JNE,Aceh,JNE_Aceh.csv,3 tahun yang lalu,2022-01-19 21:52:17.056259
3,5,Good delivery service for customers in this country.,2 years ago,JNE,Aceh,JNE_Aceh.csv,2 tahun yang lalu,2023-01-19 21:52:17.056259
4,5,Good service,3 years ago,JNE,Aceh,JNE_Aceh.csv,3 tahun yang lalu,2022-01-19 21:52:17.056259
...,...,...,...,...,...,...,...,...
2691,1,Lambaaaaaat!!,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu,2024-07-22 21:52:17.059257
2692,5,👌👌👌 …,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu,2024-07-22 21:52:17.059257
2693,5,24 jam,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu,2024-07-22 21:52:17.059257
2694,4,Ya,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu,2024-07-22 21:52:17.059257


In [37]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1935 entries, 0 to 2695
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Rating           1935 non-null   int64         
 1   Reviews          1935 non-null   object        
 2   Date             1935 non-null   object        
 3   Company          1935 non-null   object        
 4   Province         1935 non-null   object        
 5   Source           1935 non-null   object        
 6   Date_translated  1935 non-null   object        
 7   parsed_date      1935 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 136.1+ KB


In [38]:
final_df

,Rating,Reviews,Date,Company,Province,Source,Date_translated,parsed_date
0,5,"Located at the Padang Old City area, just around the corner from Siti Nurbaya Bridge across Batang Anai River. JNE is one of many delivery services in Indonesia. Giving good services, the business entity continues to grow.",9 months ago,JNE,Aceh,JNE_Aceh.csv,9 bulan yang lalu,2024-04-23 21:52:17.056259
1,1,"Bad service given to customers, my own experience taking 1 day shipment service and had to pay quite big amount of money. I asked over and over to make sure my package arrived in 1 day as it contained food and JNE staffs answer so sure it …",4 years ago,JNE,Aceh,JNE_Aceh.csv,4 tahun yang lalu,2021-01-19 21:52:17.056259
2,5,"Nice Place, they're working on rebuild their office, great people and services",3 years ago,JNE,Aceh,JNE_Aceh.csv,3 tahun yang lalu,2022-01-19 21:52:17.056259
3,5,Good delivery service for customers in this country.,2 years ago,JNE,Aceh,JNE_Aceh.csv,2 tahun yang lalu,2023-01-19 21:52:17.056259
4,5,Good service,3 years ago,JNE,Aceh,JNE_Aceh.csv,3 tahun yang lalu,2022-01-19 21:52:17.056259
...,...,...,...,...,...,...,...,...
2691,1,Lambaaaaaat!!,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu,2024-07-22 21:52:17.059257
2692,5,👌👌👌 …,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu,2024-07-22 21:52:17.059257
2693,5,24 jam,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu,2024-07-22 21:52:17.059257
2694,4,Ya,6 months ago,JNE,Sumatera Utara,JNE_Sumatera Utara.csv,6 bulan yang lalu,2024-07-22 21:52:17.059257
